In [5]:
# execute as needed
# !pip install -r requirements.txt

## Process Books

In [6]:
import sys
import warnings

sys.path.append("../")

RAW_BOOKS_FOLDER_PATH = "../raw_data/books/"
PROCESSED_BOOKS_FOLDER_PATH = "../processed_data/books/"

warnings.filterwarnings('ignore')

In [7]:
import os
import fitz
from modules.embedding_tracking import add_new_file, file_exists

available_files = os.listdir(RAW_BOOKS_FOLDER_PATH)

for i in available_files:
    print(f"Processing {i}")
    
    processed_fname = ".".join(i.split(".")[:-1])+".txt"
    print(processed_fname)
    
    if file_exists(processed_fname):
        print(f"Skipping {i} as it has already been processed.")
        continue

    text = ""
    if i.endswith(".pdf"):
        fname = os.path.join(RAW_BOOKS_FOLDER_PATH, i)
        pdf_document = fitz.open(fname)
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            page_text = page.get_text()
            # Fix spaces between words
            page_text = " ".join(page_text.replace("\n", " ").split())
            text += page_text + "\n"
        pdf_document.close()
            
    else:
        # invalid file types
        print(f"Skipping {i} as it is not a valid file.")
        continue
    
    with open(os.path.join(PROCESSED_BOOKS_FOLDER_PATH, processed_fname), "w", encoding="utf-8") as f:
        f.write(text)
        add_new_file(processed_fname)
        
    print(f"Finished processing {i}.")

Processing .DS_Store
.txt
Tracker file is empty or corrupted.
Skipping .DS_Store as it is not a valid file.
Processing Sunrise on the Reaping.pdf
Sunrise on the Reaping.txt
Tracker file is empty or corrupted.
Tracker file is empty or corrupted.
Finished processing Sunrise on the Reaping.pdf.
Processing All Fours (Miranda July).pdf
All Fours (Miranda July).txt
Finished processing All Fours (Miranda July).pdf.


### Store to storage as vectors

In [17]:
# https://medium.com/the-ai-forum/semantic-chunking-for-rag-f4733025d5f5

from modules import accessor, embedding_tracking
from langchain.text_splitter import RecursiveCharacterTextSplitter
from nltk.tokenize import sent_tokenize
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import json
import numpy as np

nltk.download("punkt")

filenames = os.listdir(PROCESSED_BOOKS_FOLDER_PATH)
embedding_functions = list(accessor.EMBEDDING_FUNCTIONS.keys())
stitch_title = "TITLE: {}\n"

for filename in filenames:
    with open(os.path.join(PROCESSED_BOOKS_FOLDER_PATH, filename), "r", encoding="utf-8") as file:
        text = file.read()
        print(f"Tokenizing {filename}")
        
        for embedding_function in embedding_functions:
            # Fixed size chunk text embedding
            suffix = "512_chunks"
            if not embedding_tracking.is_file_processed(filename, "{}_{}".format(embedding_function, suffix)):
                chunked_texts = [stitch_title.format(filename) + text[i:i+512] for i in range(0, len(text), 512)]
                try:
                    ids = accessor.insert(
                        data=chunked_texts,
                        embedding_func=embedding_function,
                        custom_suffix=suffix,
                    )
                    embedding_tracking.mark_file_processed(filename, "{}_{}".format(embedding_function, suffix))
                except Exception as e:
                    print(f"Error storing {suffix} for {filename}: {e}")
            else:
                print(f"Skipping {filename} for {embedding_function} as it has already been processed with suffix {suffix}.")
                
            # Recursive character text splitting
            suffix = "recursive_chunks"
            if not embedding_tracking.is_file_processed(filename, "{}_{}".format(embedding_function, suffix)):
                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=512,
                    chunk_overlap=100,
                    length_function=len,
                )
                chunks = text_splitter.split_text(text)
                chunks = [stitch_title.format(filename) + chunk for chunk in chunks]
                try:
                    ids = accessor.insert(
                        data=chunks,
                        embedding_func=embedding_function,
                        custom_suffix=suffix,
                    )
                    embedding_tracking.mark_file_processed(filename, "{}_{}".format(embedding_function, suffix))
                except Exception as e:
                    print(f"Error storing {suffix} for {filename}: {e}")
                    
            # Separator text splitting
            suffix = "separator_chunks"
            if not embedding_tracking.is_file_processed(filename, "{}_{}".format(embedding_function, suffix)):
                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=512,
                    chunk_overlap=100,
                    length_function=len,
                    separators=["CHAPTER", "\n\n", "\n", " ", ""],
                )
                chunks = text_splitter.split_text(text)
                chunks = [stitch_title.format(filename) + chunk for chunk in chunks]
                try:
                    ids = accessor.insert(
                        data=chunks,
                        embedding_func=embedding_function,
                        custom_suffix=suffix,
                    )
                    embedding_tracking.mark_file_processed(filename, "{}_{}".format(embedding_function, suffix))
                except Exception as e:
                    print(f"Error storing {suffix} for {filename}: {e}")
            
            # Semantic chunking
            suffix = "semantic_chunks"
            if not embedding_tracking.is_file_processed(filename, "{}_{}".format(embedding_function, suffix)):
                sentences = sent_tokenize(text)
                threshold = 0.7 # can be adjusted
                chunks = []
                curr_chunk = [stitch_title.format(filename), sentences[0]]
                for i in range(0, len(sentences)-1):
                    
                    embeddings_1 = accessor.EMBEDDING_FUNCTIONS.get(embedding_function).embed_query(sentences[i])
                    embeddings_2 = accessor.EMBEDDING_FUNCTIONS.get(embedding_function).embed_query(sentences[i+1])
                    
                    emb_1 = np.array(embeddings_1)
                    emb_2 = np.array(embeddings_2)

                    if np.linalg.norm(emb_1) == 0 or np.linalg.norm(emb_2) == 0:
                        continue

                    try:
                        similarity = cosine_similarity([emb_1], [emb_2])[0][0]
                    except Exception as e:
                        print(f"Error calculating similarity for {sentences[i]} and {sentences[i+1]}: {e}")
                        continue

                    if similarity > threshold:
                        chunks.append(curr_chunk)
                        curr_chunk = [stitch_title.format(filename), sentences[i+1]]
                        continue
                    
                    curr_chunk.append(sentences[i+1])
                
                try:
                    ids = accessor.insert(
                        data=[" ".join(chunk) for chunk in chunks],
                        embedding_func=embedding_function,
                        custom_suffix=suffix,
                    )
                    embedding_tracking.mark_file_processed(filename, "{}_{}".format(embedding_function, suffix))
                except Exception as e:
                    print(f"Error storing {suffix} for {filename}: {e}")
            
    print(f"Finished processing {filename} for embeddings.")
    
# LLM based chunking
suffix = "llm_chunked"
llm_chunked_fnames = ["all_fours_txt_chunks.json", "sunrise_on_the_reaping_chunks.json"]
for fname in llm_chunked_fnames:
    embedding_tracking.add_new_file(fname)
    
for embedding_function in embedding_functions:
    for fname in llm_chunked_fnames:
        if not embedding_tracking.is_file_processed(fname, "{}_{}".format(embedding_function, suffix)):
            try:
                with open(os.path.join("../llm_chunked_data", fname), "r", encoding="utf-8") as f:
                    chunks = json.load(f)
                
                chunks = [stitch_title.format(fname) + chunk for chunk in chunks]
                ids = accessor.insert(
                    data=chunks,
                    embedding_func=embedding_function,
                    custom_suffix=fname,
                )
                embedding_tracking.mark_file_processed(fname, "{}_{}".format(embedding_function, suffix))
            except Exception as e:
                print(f"Error storing {suffix} for {fname}: {e}")
        else:
            print(f"Skipping {fname} for {embedding_function} as it has already been processed with suffix {suffix}.")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yudhistiraonggowarsito/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenizing All Fours (Miranda July).txt
Skipping All Fours (Miranda July).txt for nomic-embed-text as it has already been processed with suffix 512_chunks.
Skipping All Fours (Miranda July).txt for mxbai-embed-large as it has already been processed with suffix 512_chunks.
Inserted data with IDs: ['64370857-e9ca-4a39-b3d1-aaceb8671478', '47f4f4c0-673f-421c-9b9a-6dff377dbeb4', 'cc377a3b-d292-45db-b1d0-ea4fc862710e', 'ec035d1a-1e0b-4ced-82b2-f0717e253ad6', '9529b113-0292-454a-b3a3-b85cbbe948a3', '24f0a8d1-7629-4cb9-8817-a451c99510b7', '20b86b3c-d348-4a07-83ed-c3a654bf4e1f', '15b753f8-b91a-4b2f-869b-7802b8e39510', '9e65c589-ff4a-4da1-a25c-6e66d32c4244', '907af3c0-c8dd-43b9-8fe1-a1880f9ec6de', '2f9615c7-ff12-43a1-9645-c2a5c53d35cf', 'eeca1e3d-97c4-4cbd-a8e1-5ebd46f7a3bd', 'e98a8eab-5bf5-4af9-97f4-eb930f94f78a', '0b727c0e-f222-416d-9b58-04f8b2b485c1', 'aafcde2b-ce4f-43c9-80dd-58dab92d8727', '6ca526cf-a5ca-4ba0-bd59-1419c07db402', 'c55c492d-0338-40bb-9cf1-0a72604094ff', 'f1ef2090-d95f-4542-8f

KeyboardInterrupt: 

In [22]:
available_collections = accessor.list_collections()
print(f"Available collections: {available_collections}")

documents = accessor.get(
    query="In the novel 'Sunrise on the Reaping', what is the final 'poster' Haymitch creates in the arena?",
    embedding_func="bge-m3",
    custom_suffix="semantic_chunks",
    n_results=10
)

for i in documents:
    print(i.page_content)
    print("-------------------------")

Available collections: ['nomic-embed-text_collection_separator_chunks', 'bge-m3_collection_512_chunks', 'bge-m3_collection_llm_chunked', 'bge-m3_collection_semantic_chunks', 'mxbai-embed-large_collection_semantic_chunks', 'bge-m3_collection_recursive_chunks', 'bge-m3_collection_sunrise_on_the_reaping_chunks.json', 'mxbai-embed-large_collection_llm_chunked', 'mxbai-embed-large_collection_all_fours_txt_chunks.json', 'nomic-embed-text_collection_semantic_chunks', 'mxbai-embed-large_collection_recursive_chunks', 'mxbai-embed-large_collection_512_chunks', 'bge-m3_collection_all_fours_txt_chunks.json', 'nomic-embed-text_collection_sunrise_on_the_reaping_chunks.json', 'nomic-embed-text_collection_all_fours_txt_chunks.json', 'bge-m3_collection_separator_chunks', 'mxbai-embed-large_collection_sunrise_on_the_reaping_chunks.json', 'nomic-embed-text_collection_recursive_chunks', 'mxbai-embed-large_collection_separator_chunks', 'nomic-embed-text_collection_llm_chunks', 'nomic-embed-text_collection_